In [ ]:
import os
import csv
import time
import folium
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import KMeans
from geopy.distance import geodesic
from typing import List, Dict, Tuple, Generator

In [ ]:
np.random.seed(3)
random.seed(3)

In [ ]:
####### Global performance def #######

"""

The purpose of this code is to define a decorator function called measure_time that 
can be used to measure the execution time of other functions. By applying this decorator 
to a function, you can easily track how long it takes to execute.

When the measure_time decorator is applied to a function, it wraps the original 
function with a wrapper function. The wrapper function checks if the measurement is enabled. 
If it is, the wrapper function records the start time, calls the original function, 
records the end time, calculates the execution time, and prints the results. 
Finally, it returns the result of the original function.


"""

def measure_time(func):
    # Get the enabled value outside the wrapper
    enabled = measure_time.enabled  

    def wrapper(*args, **kwargs):
        # ensure we're using the latest settings
        enabled = measure_time.enabled  
        if enabled:
            
            start_time = time.time()  
            # Call the original function with the provided arguments
            result = func(*args, **kwargs)  
            end_time = time.time()  
            execution_time = end_time - start_time 
            
            print(f" the Function: {func.__name__} took: {execution_time} seconds") 
            return result  # Return the result of the decorated function
        else:
            # If the decorator is disabled, simply call the original function
            return func(*args, **kwargs)  

    # Return the wrapper function only if enabled, else return the original function
    return wrapper if enabled else func


####### ####### ####### ####### #######

In [ ]:
####### Global ploting system #######

"""

The purpose of this code is to define a function called plot_cities that creates a map 
with markers and lines to visualize city data. The function takes 
two parameters: city_data, which contains the information about the cities to be 
plotted, and bound (optional), which specifies the order of cities to be plotted.

The plot_cities function contains several helper functions:

* arrange_cities_nearest: This function arranges the cities in city_data in the order 
specified by path. It sorts the cities based on their index in the path list, ensuring 
that the cities are plotted in the desired order.

* create_map_center: This function extracts the coordinates of the first city in 
city_data and returns them as the center of the map.

* create_custom_icon_style: This function defines a custom icon style for the markers 
on the map. It sets the background color, text color, border radius, padding, font weight, 
and font size.

* add_marker_with_icon: This function adds a marker with a custom icon to the map. 
It takes the coordinates, name, icon style, and index as parameters and creates a marker 
with a numbered icon at the specified coordinates.

* draw_line_between_cities: This function draws a line between two cities on the map. 
It takes the coordinates of the current city, the coordinates of the next city, the names 
of both cities, and the index as parameters. It creates a polyline connecting the two cities with 
a tooltip indicating the route number and the city names.

"""

color = ["Green", "Blue", "Yellow", "Orange", "Purple", "Pink", "Brown", "Gray", "Black", "White"]

def plot_cities(city_data, bound=None, color='blue', map_obj=None):
    
    def arrange_cities_nearest(city_data, path):
        city_names = city_data[:, -1]
        sorted_indices = np.argsort([path.index(city) for city in city_names])
        sorted_city_data = city_data[sorted_indices]
        return np.vstack((sorted_city_data, sorted_city_data[0]))

    def create_map_center():
        return [float(city_data[0][2]), float(city_data[0][3])]

    def create_custom_icon_style():
        return """
            background-color: #ff5959;
            color: #ffffff;
            border-radius: 100%;
            padding: 20%;
            text-align: center;
            font-weight: bold;
            font-size: auto;
        """

    def add_marker_with_icon(coords, name, icon_style, i):
        folium.Marker(
            coords,
            popup=name,
            icon=folium.DivIcon(
                icon_size=(24, 24),
                icon_anchor=(12, 12),
                html='<div style="{}">{}</div>'.format(icon_style, i + 1)
            )
        ).add_to(map_obj)

    def draw_line_between_cities(coords, next_coords, name, next_city, i):
        folium.PolyLine(
            [coords, next_coords],
            color=color,
            weight=2.5,
            opacity=1.0,
            tooltip='Route {}: {} -> {}'.format(i + 1, name, next_city[4])
        ).add_to(map_obj)

    # Create a map if not provided
    if map_obj is None:
        map_center = create_map_center()
        map_obj = folium.Map(location=map_center, zoom_start=6)

    if bound:
        city_data = arrange_cities_nearest(city_data, bound)
        # Iterate over the city data and plot markers and lines
        for i in range(len(city_data) - 1):
            city = city_data[i]
            coords = [float(city[2]), float(city[3])]
            name = city[4]

            icon_style = create_custom_icon_style()

            add_marker_with_icon(coords, name, icon_style, i)

            next_city = city_data[i + 1]
            next_coords = [float(next_city[2]), float(next_city[3])]

            draw_line_between_cities(coords, next_coords, name, next_city, i)
    else:
        # Iterate over the city data and create a folium marker for each city
        for city in city_data:
            folium.Marker(
                location=[float(city[2]), float(city[3])],
                popup=city[4]
            ).add_to(map_obj)

    # Display the map
    return map_obj

####### ####### ####### ####### #######

In [ ]:
####### Global notebook configs #######

# Toggle for enabling/disabling the 
# decorator
measure_time.enabled = True 
if measure_time.enabled:
    print("* measure_time is enabled ")

# specify the folder path and files name 
dataset_file_path = os.path.join('../datasets', 'cities.csv')
print(f"* the selected dataset is located at: {dataset_file_path}")

# select the nb of cities you want from the dataset
np_of_cities = 200
print(f"* {np_of_cities} will be used from the dataset")

# select the number of truck you wish to divide the workload
truck_nb = 5
print(f"* {truck_nb} will be used to deliver the goods")

# choose an average speed that suit your needs (here 51.3 km/h)
average_speed = 51.3
print(f"* the average speed is : {average_speed} km/h")

# tabu related conf 
num_iterations = 100
tabu_list_size = 100
print(f"* the tabu list size is : {tabu_list_size} and it will iterate {num_iterations} times")

# multistart related conf
num_starts = 10
print(f"* multistart factor : {num_starts}")

####### ####### ####### ####### #######

# City Generator 

this part contains the folowing logic: we first retrieve data from a dataset and later construct a sample from it that contain the cities name, the ZIP Code, the population count and longitude|latitude 

<u>non-optimized</u>

In [ ]:
@measure_time
def read_csv_to_tuple(filename: str):
    with open(filename, "r", encoding='ISO-8859-1') as fh:  # Open the file in read mode
        # Create a CSV reader object with delimiter ';'
        reader = csv.reader(fh, delimiter=';')  
        # Skip the header row
        next(reader, None)  
        # Convert the remaining rows to a tuple
        cities = tuple(reader)  
    return cities  # Return the tuple


@measure_time
def sample_N_from_tuple(cities: tuple, size: int = None):
    totalRows = len(cities)
    # If size is not specified or greater than totalRows Return 
    # an empty tuple
    if size is None or size > totalRows:  
        return ()
    # Return a random sample of 'size' elements from the tuple
    return random.sample(cities, size)  

In [ ]:
citiesTuple = read_csv_to_tuple(dataset_file_path)
citiesSample = sample_N_from_tuple(citiesTuple, np_of_cities)

<u>optimized</u>

- Reads the CSV file using the pandas library's read_csv function.
    - Efficient CSV reading: The optimized version uses pandas' read_csv function, which is highly optimized for reading CSV files. It takes advantage of optimized file parsing algorithms and efficient memory management, resulting in faster file reading compared to the line-by-line reading in the non-optimized version.

- Converts the DataFrame to a tuple of lists and then to a tuple.
    - In the non-optimized version, each row from the CSV file is converted to a tuple individually. In the optimized version, pandas converts the entire DataFrame to a tuple of lists in one operation, which is more efficient and faster.
    
- Uses pandas and NumPy functions for sampling instead of the random.sample function.
    - The non-optimized version uses the random.sample function to sample elements from the tuple. In the optimized version, NumPy's random.choice function is used, which is implemented in optimized C code and performs faster random sampling.
    - The optimized version uses pandas' iloc function to extract the sampled data based on the selected indices. This indexing operation is optimized in pandas and provides faster access to the desired rows.

In [ ]:
@measure_time
def read_csv_to_tuple(filename: str):
    # Read the CSV file using pandas
    df = pd.read_csv(filename, delimiter=';', encoding='ISO-8859-1')
    # Convert the DataFrame to a tuple of lists and then to a tuple
    cities = tuple(df.values.tolist())  
    return cities


@measure_time
def sample_N_from_tuple(cities: tuple, size: int = None):
    # Create a DataFrame from the tuple of lists
    df = pd.DataFrame(list(cities))
    # Get the total number of rows in the DataFrame
    totalRows = len(df)
    
    # If size is not specified or greater than totalRows
    # Return an empty tuple
    if size is None or size > totalRows:  
        return ()
    
    # Randomly select 'size' indices without replacement
    indices = np.random.choice(totalRows, size, replace=False)
    # Extract the sampled data based on the selected indices
    sampled_data = df.iloc[indices].values.tolist()  
    # Return the sampled data as a NumPy array
    return np.array(sampled_data)  

In [ ]:
citiesTuple = read_csv_to_tuple(dataset_file_path)
citiesSample = sample_N_from_tuple(citiesTuple, np_of_cities)

<u>the actual output of the city generator section</u>

In [ ]:
# disable performance profiling for this section 
measure_time.enabled = False

citiesTuple = read_csv_to_tuple(dataset_file_path)
citiesSample = sample_N_from_tuple(citiesTuple, np_of_cities)

# display the map with the selected cities
plot_cities(citiesSample)

# location generator 
The purpose of this staged is to generate a series of city names along with their respective longitude and latitude coordinates. It achieves this by extracting the relevant information from a given list of city data

In [ ]:
# enable performance profiling for this section 
measure_time.enabled = True

<u>non-optimized</u>

In [ ]:
@measure_time
def create_location_generator(citiesSample: List[List[str]]) -> Dict[str, Tuple[float, float]]:
    # Create an empty dictionary to store the location data
    tmp = {}  
    
    for city in citiesSample:  # Iterate over each city in citiesSample
        city_name = city[4]  # Get the city name from the city data
        longitude = float(city[3])  # Get the longitude from the city data and convert it to float
        latitude = float(city[2])  # Get the latitude from the city data and convert it to float
        tmp[city_name] = (longitude, latitude)  # Store the longitude and latitude as a tuple in the dictionary
    
    # Return the dictionary containing the location data
    return tmp

In [ ]:
location = create_location_generator(citiesSample)

<u>optimized</u>

- Uses NumPy array indexing to extract city names, longitudes, and latitudes from the citiesSample list in one operation.
    - The optimized version leverages NumPy's array indexing and vectorized operations to extract the necessary data from the citiesSample list. This allows for faster and more efficient data extraction compared to the iterative approach in the non-optimized version.
- Converts the longitudes and latitudes to float using NumPy's astype function.
    - In the non-optimized version, the conversion to float is performed individually for each longitude and latitude. In the optimized version, NumPy's astype function is applied to the entire arrays of longitudes and latitudes in one operation. This bulk conversion is more efficient and faster.
- Utilizes the zip function and generator syntax (yield from) to create a generator that yields tuples of city names and corresponding longitude-latitude pairs.
    - The optimized version uses a generator and the yield from syntax to produce the desired output. Generators provide a memory-efficient way to produce values on-the-fly, as opposed to constructing and returning a complete dictionary in the non-optimized version. This can improve performance, especially when dealing with large datasets.


In [ ]:
@measure_time
def create_location_generator(citiesSample: List[List[str]]) -> Generator[Tuple[str, Tuple[float, float]], None, None]:
    # Extract city names from citiesSample using NumPy array indexing
    city_names = np.array(citiesSample)[:, 4]
    # Extract longitudes and convert them to float using NumPy array indexing
    longitudes = np.array(citiesSample)[:, 3].astype(float)
    # Extract latitudes and convert them to float using NumPy array indexing
    latitudes = np.array(citiesSample)[:, 2].astype(float)

    yield from zip(city_names, zip(longitudes, latitudes))

In [ ]:
location = create_location_generator(citiesSample)

<u>the actual output of the location generator section</u>

In [ ]:
# disable performance profiling for this section 
measure_time.enabled = False

for city_name, coordinates in create_location_generator(citiesSample):
    print(f'{city_name}: {coordinates}')

# time matrix generator

this part calculate a time matrix for a set of cities based on their geographic coordinates. 

In [ ]:
# enable performance profiling for this section 
measure_time.enabled = True

<u>non-optimized</u>

In [ ]:
@measure_time
def calculate_time_matrix(generator) -> Dict[str, Dict[str, float]]:
    time_matrix = {}  # Create an empty dictionary to store the time matrix
    city_coords = []  # Create an empty list to store city names and coordinates
    
    # Iterate over each city name and coordinates from the generator and 
    # append the city name and coordinates as a tuple to city_coords
    for city_name, coordinates in generator:
        city_coords.append((city_name, coordinates)) 
    
    # Iterate over the city name and coordinates using enumerate
    for i, (city1, coords1) in enumerate(city_coords):
        
        # Create an empty dictionary for each city in the time matrix
        time_matrix[city1] = {}
        
        # Iterate over the city name and coordinates again
        for j, (city2, coords2) in enumerate(city_coords):  
            if i == j:
                # Set the time between a city and itself to 0.0
                time_matrix[city1][city2] = 0.0 
            else:
                # Calculate the geodesic distance between two coordinates
                distance = geodesic(coords1, coords2).kilometers
                # Store the time in the time matrix
                time_matrix[city1][city2] = distance / average_speed 
    
    return time_matrix

In [ ]:
time_matrix = calculate_time_matrix(
    create_location_generator(citiesSample)
)

<u>optimized</u>

- The optimized version directly stores the city coordinates in a dictionary (city_coords), eliminating the need for additional data structures like the city_coords list in the non-optimized version. This reduces memory usage and unnecessary operations, resulting in improved performance.
- The optimized version utilizes NumPy's vectorized operations to calculate times between pairs of coordinates. By converting the coordinates to a NumPy array and using broadcasting, the calculations can be performed efficiently in parallel, leading to significant speed improvements.
- Instead of constructing an empty dictionary for each city, the optimized version creates a square matrix (times) with zeros to store the time values. This allows for efficient indexing and updating of the times using NumPy operations.
- The optimized version converts the times matrix to a pandas DataFrame, which provides efficient indexing capabilities and convenient conversion to a dictionary. This avoids nested loops and dictionary updates in the non-optimized version, resulting in improved performance.

<u>the actual output of the time matrix generator section</u>

In [ ]:
# disable performance profiling for this section 
measure_time.enabled = False

calculate_time_matrix(
    create_location_generator(citiesSample)
)

# tabu algorithm

In [ ]:
def pprint_the_output(best_time, best_route): 
    print(f"Best time: {best_time} h")
    print("Best Route:", ' -> '.join(best_route))

In [ ]:
def tabu_search(time_matrix, num_iterations, tabu_list_size, start_town=None, progress_enable = True):
    # Initialize the tabu list as an empty set
    tabu_list = set()
    # Generate an initial random solution
    if start_town is None:
        best_route = list(time_matrix.keys())
        np.random.shuffle(best_route)
    else:
        best_route = list(time_matrix.keys())
        best_route.remove(start_town)
        best_route.insert(0, start_town)
    best_time = calculate_total_time(best_route, time_matrix)

    # Create a progress bar using tqdm
    if progress_enable: 
        progress_bar = tqdm(
            total=num_iterations, 
            desc="Tabu Search", 
            unit="iteration", 
        )

    # Start the iterations
    for _ in range(num_iterations):
        # Find the best neighboring solution
        neighbors = generate_neighbors(best_route, tabu_list)
        
        # [hack] Check if neighbors list is empty 
        if not neighbors:
            if progress_enable: 
                progress_bar.update(num_iterations - progress_bar.n)  # Force progress to 100%
            continue
        
        best_neighbor = min(neighbors, key=lambda x: calculate_total_time(x, time_matrix))

        # Update the best solution if the neighbor is an improvement
        neighbor_time = calculate_total_time(best_neighbor, time_matrix)
        if neighbor_time < best_time:
            best_route = best_neighbor
            best_time = neighbor_time

        # Add the best neighbor to the tabu list
        tabu_list.add(tuple(best_neighbor))
        # Remove the oldest solution from the tabu list if it exceeds the tabu list size
        if len(tabu_list) > tabu_list_size:
            tabu_list.remove(next(iter(tabu_list)))

        # Update the progress bar
        if progress_enable: 
            progress_bar.update(1)

    # Append the first town to the best route to complete the cycle
    best_route.append(best_route[0])
    best_time += time_matrix[best_route[-2]][best_route[0]]

    # Close the progress bar
    if progress_enable: 
        progress_bar.close()

    return best_route, best_time


def calculate_total_time(route, time_matrix):
    total_time = 0.0
    num_cities = len(route)
    for i in range(num_cities - 1):
        current_city = route[i]
        next_city = route[i + 1]
        total_time += time_matrix[current_city][next_city]
    return total_time


def generate_neighbors(route, tabu_list):
    neighbors = []
    num_cities = len(route)
    for i in range(1, num_cities):
        for j in range(i + 1, num_cities):
            neighbor = route[:i] + route[i:j][::-1] + route[j:]
            if tuple(neighbor) not in tabu_list:
                neighbors.append(neighbor)
    return neighbors

In [ ]:
time_matrix = calculate_time_matrix(
    create_location_generator(citiesSample)
)

best_route, best_time = tabu_search(time_matrix, num_iterations, tabu_list_size)

pprint_the_output(best_time, best_route)

In [ ]:
plot_cities(citiesSample, best_route)

# tabu algorithm with traffic constraints

In [ ]:
"""

The traffic matrix generator applies a random factor to the times 
in order to simulate the increased time it would take to travel from 
city A to city B due to traffic conditions. 

"""

def generate_traffic_matrix(time_matrix):
    traffic_matrix = {}

    for city_a, times in time_matrix.items():
        traffic_matrix[city_a] = {}

        for city_b, time in times.items():
            if city_a == city_b:
                traffic_matrix[city_a][city_b] = 1.0  # Assuming no traffic within the same city
            else:
                traffic_matrix[city_a][city_b] = np.random.uniform(1.0, 1.5) * time
    
    return traffic_matrix

In [ ]:
time_matrix = calculate_time_matrix(
    create_location_generator(citiesSample)
)

traffic_matrix = generate_traffic_matrix(time_matrix)

best_route, best_time = tabu_search(traffic_matrix, num_iterations, tabu_list_size)

pprint_the_output(best_time, best_route)

In [ ]:
plot_cities(citiesSample, best_route)

# tabu algorithm with nb truck constraints

In [ ]:
def split_into_trucks(time_matrix, num_trucks):
    # Convert the time matrix to a numpy array
    towns = list(time_matrix.keys())
    time_array = np.array([[time_matrix[town1][town2] for town2 in towns] for town1 in towns])
    
    # Apply k-means clustering
    kmeans = KMeans(n_clusters=num_trucks, random_state=0, n_init=10).fit(time_array)
    labels = kmeans.labels_
    
    # Find the index of the first town in the time matrix
    first_town_index = towns.index(towns[0])
    
    # Create time matrices for each truck
    trucks = []
    for i in range(num_trucks):
        # Filter towns based on the label or if it's the first town
        truck_towns = [towns[j] for j in range(len(towns)) if labels[j] == i or j == first_town_index]
        truck_time_matrix = {town: {truck_town: time_matrix[town][truck_town] for truck_town in truck_towns} for town in truck_towns}
        trucks.append(truck_time_matrix)
    
    return trucks

def filter_cities_by_truck(citiesSample, truck):
    town_list = list(truck.keys())
    filtered_cities = [city for city in citiesSample if city[4] in town_list]
    return np.array(filtered_cities)


def optimize_trucks(trucks, map_obj, use_traffic_matrix=False):
    for i, truck in enumerate(trucks):
        if use_traffic_matrix:
            truck = generate_traffic_matrix(truck)
        best_route, best_time = tabu_search(truck, num_iterations, tabu_list_size, start_town=next(iter(truck)))
            
        print(f"The truck n: {i}")
        pprint_the_output(best_time, best_route)
        plot_cities(filter_cities_by_truck(citiesSample, truck), best_route, map_obj=map_obj, color=color[i])
    
    return map_obj

In [ ]:
trucks = split_into_trucks(time_matrix, truck_nb)

map_center = [float(citiesSample[0][2]), float(citiesSample[0][3])]
map_obj = folium.Map(location=map_center, zoom_start=6)


optimize_trucks(trucks, map_obj)

# tabu algorithm with nb truck and  traffic constraints

In [ ]:
trucks = split_into_trucks(time_matrix, truck_nb)

map_center = [float(citiesSample[0][2]), float(citiesSample[0][3])]
map_obj = folium.Map(location=map_center, zoom_start=6)

optimize_trucks(trucks, map_obj, use_traffic_matrix=True)

# dummy multi-start

In [ ]:
from tqdm import tqdm

def multi_start_optimization(time_matrix, truck_nb, map_obj, num_starts, use_traffic_matrix=False):
    best_routes = []
    best_times = []
    total_times = []
    trucks_list = []

    for i in tqdm(range(num_starts), desc="multistart"):
        time_matrix_shuffled = list(time_matrix.items())
        np.random.shuffle(time_matrix_shuffled)
        time_matrix_shuffled = dict(time_matrix_shuffled)
        
        trucks = split_into_trucks(time_matrix_shuffled, truck_nb)
        trucks_list.append(trucks)
        
        start_routes = []
        start_times = []

        for i, truck in enumerate(trucks):
            if use_traffic_matrix:
                truck = generate_traffic_matrix(truck)
            route, time = tabu_search(truck, num_iterations=100, tabu_list_size=100, start_town=next(iter(truck)), progress_enable=False)

            start_routes.append(route)
            start_times.append(time)

        best_routes.append(start_routes)
        best_times.append(start_times)
        total_time = sum(start_times)
        total_times.append(total_time)

    sorted_starts = sorted(range(num_starts), key=lambda k: total_times[k], reverse=True)
        
    for i, truck in enumerate(trucks_list[sorted_starts[-1]]):
        route = [lst for lst in best_routes[sorted_starts[-1]] if list(truck.keys())[1] in lst][0]
        plot_cities(filter_cities_by_truck(citiesSample, truck), route, map_obj=map_obj, color=color[i])
    
    print(f"the best route is : {best_routes[sorted_starts[-1]]} \n and it will take {total_times[sorted_starts[-1]]} h")
    return map_obj

In [ ]:
map_center = [float(citiesSample[0][2]), float(citiesSample[0][3])]
map_obj = folium.Map(location=map_center, zoom_start=6)

multi_start_optimization(time_matrix, truck_nb, map_obj, num_starts)

# multistart with //

In [ ]:
import concurrent.futures

def multi_start_optimization(time_matrix, truck_nb, map_obj, num_starts, use_traffic_matrix=False, max_threads=4):
    best_routes = []
    best_times = []
    total_times = []
    trucks_list = []

    def optimize_start(start_index):
        time_matrix_shuffled = list(time_matrix.items())
        np.random.shuffle(time_matrix_shuffled)
        time_matrix_shuffled = dict(time_matrix_shuffled)

        trucks = split_into_trucks(time_matrix_shuffled, truck_nb)
        trucks_list.append(trucks)

        start_routes = []
        start_times = []

        for i, truck in enumerate(trucks):
            if use_traffic_matrix:
                truck = generate_traffic_matrix(truck)

            route, time = tabu_search(truck, num_iterations=100, tabu_list_size=100, start_town=next(iter(truck)), progress_enable=False)

            start_routes.append(route)
            start_times.append(time)
            
            progress_bar.update(1)  # Increment the common progress bar

        best_routes.append(start_routes)
        best_times.append(start_times)
        total_time = sum(start_times)
        total_times.append(total_time)

    # Create a single common progress bar
    progress_bar = tqdm(total=num_starts * truck_nb, desc="parallelised multistart", position=0)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = [executor.submit(optimize_start, i) for i in range(num_starts)]
        concurrent.futures.wait(futures)

    progress_bar.close()  # Close the common progress bar

    sorted_starts = sorted(range(num_starts), key=lambda k: total_times[k], reverse=True)

    for i, truck in enumerate(trucks_list[sorted_starts[-1]]):
        route = [lst for lst in best_routes[sorted_starts[-1]] if list(truck.keys())[1] in lst][0]
        plot_cities(filter_cities_by_truck(citiesSample, truck), route, map_obj=map_obj, color=color[i])

    print(f"The best route is: {best_routes[sorted_starts[-1]]}\n and it will take {total_times[sorted_starts[-1]]} h")
    return map_obj

In [ ]:
map_center = [float(citiesSample[0][2]), float(citiesSample[0][3])]
map_obj = folium.Map(location=map_center, zoom_start=6)

multi_start_optimization(time_matrix, truck_nb, map_obj, num_starts)